<a href="https://colab.research.google.com/github/eldercamposds/LangChain/blob/teste/ChatPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/LangChain
!pip install -r requirements.txt

/content/drive/MyDrive/Colab Notebooks/LangChain
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
Reason for being yanked: The Windows wheels, under some conditions, caused segfaults in unrelated user code.  Due to this we deleted the Windows wheels to prevent these segfaults, however this caused greater disruption as pip then began to try (and fail) to build 3.9.1 from the sdist on Windows which impacted far more users.  Yanking the whole release is the only t

# Setup

In [2]:
import numpy as np
from pinecone import Pinecone
import yaml
import os

In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/LangChain/config.yaml', 'r') as config_file:
  config = yaml.safe_load(config_file)

os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']

In [5]:
pinecone_client = Pinecone()

#Pinecone

In [7]:
indices = pinecone_client.list_indexes() #retorna lista de todos os index do pinecone
for index in indices:
  index_name = index['name']
  print(f"Index: {index_name}")
  print(pinecone_client.describe_index(index_name))

Index: nlp
{'dimension': 2048,
 'host': 'nlp-ptq58qc.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'nlp',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [8]:
indice_name = 'nlp'

In [12]:
vetores = [np.random.normal(0,1,2048).tolist() for _ in range(5)] # criando 5 listas com distribuição normal, média 0, desvio padrão 1 e 2048 valores
ids = ["a", "b", "c", "d", "e"] #identiifcadores para os vetores
indice = pinecone_client.Index(indice_name)
indice.upsert(vectors=list(zip(ids, vetores)))#upset cria ou atualiza o indice

{'upserted_count': 5}

In [14]:
print(len(vetores))
print(vetores[4])

5
[-0.1114519018288789, 0.35760611947717097, -1.2474826263343752, 0.40583986002544725, -0.8962166499824334, -0.4527951921364008, -1.4370428448633124, -0.6268760172674962, -0.9004081146333706, -2.034608839923392, 0.9706101194995992, -0.07868382052599242, -0.8577021256958867, 1.5693445545536204, 1.2750116215752614, 0.3794474854650782, -1.3221086496389454, 0.2817609796502819, -0.7224277279633086, 2.428808256043398, -0.5936357043421548, -0.5815750659354698, 0.8291138610541352, 0.009568930866218232, 1.588445551663377, -2.074077163936485, -0.8372581898441894, -0.15483597020241371, -0.03655565373339507, -0.6570063100740888, -0.20038498125563659, 1.3386588223419817, -1.0875978842592546, -3.1642667057628255, -0.8905783929370716, 1.2865966693729018, -2.358115064984456, 0.012936130514823072, -1.3324287139651556, 1.2697684211148765, -1.3476605285946086, -1.6605830009265823, -0.8880520943204427, 0.21496798898566766, 0.9982455398137237, 0.6277363422609854, 0.5820399998001852, -0.8641950961036081, 0.

In [ ]:
print(indice.fetch(ids=["c"]))
#0.409804225 primeiro valor do indice c

In [ ]:
response = indice.fetch(ids=["c"])
if 'vctors' in response and 'c' in response['vectors']:
  retorna_valor = response['vectors']['c']['values']
  atualiza_valor = response[x + 1 for x in retorna_valor]
  indice.upset(vectors=[('c', atualiza_valor)])
  print(indice.fetch(ids=['c']))
else:
  print('Vetor não encontrado!')

In [18]:
indice.delete(ids=['d','e'])

{}

In [19]:
print(indice.fetch(ids=['d','e']))

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}


In [21]:
print(indice.describe_index_stats()) #descrição dos indices

{'dimension': 2048,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}


In [23]:
indice.upsert(vectors=list(zip(ids, vetores)), namespace="namespace1") # se não for definido o namespace será criado um namespace em branco
indice.upsert(vectors=list(zip(["x", "y", "z"],
                                [np.random.normal(0, 1, 2048).tolist() for _ in range(3)])),
                                 namespace="namespace2")# criando segundo namespace com 3 indices

{'upserted_count': 3}